In [1]:
import requests

requests.adapters.DEFAULT_RETRIES = 1000

In [2]:
import glob
import pandas as pd
from datetime import datetime
import io
import re
from cucco import Cucco
from unidecode import unidecode
import pandas as pd

In [3]:
from aip import AipNlp

""" 你的 APPID AK SK """
APP_ID = '11584634'
API_KEY = 'ulwuucx5BFFdcKX1xn3P4y2C'
SECRET_KEY = 'DV3HxNeorC4gEno1qmrW1lyxZN18B9V9'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [4]:
def remove_emoji(text):
    cucco = Cucco()
    return cucco.replace_emojis(text)

def is_ustr(in_str):
    """transfer non-chinese unicodes to utf-8"""
    out_str=''
    for i in range(len(in_str)):
        if is_uchar(in_str[i]):
            out_str=out_str+in_str[i]
        else:
            out_str=out_str+unidecode(in_str[i])
    return out_str
def is_uchar(uchar):
    """判断一个unicode是否是汉字"""
    if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
            return True
    #"""判断一个unicode是否是数字"""
    #if uchar >= u'\u0030' and uchar<=u'\u0039':
    #        return False        
    #"""判断一个unicode是否是英文字母"""
    #if (uchar >= u'\u0041' and uchar<=u'\u005a') or (uchar >= u'\u0061' and uchar<=u'\u007a'):
    #        return False
    #if uchar in ('-',',','，','。','.','>','?'):
    #        return False
    return False

In [5]:
text="""何弘恺的脚太臭了"""
result=client.sentimentClassify(text)

In [6]:
result

{'items': [{'confidence': 0.759057,
   'negative_prob': 0.891576,
   'positive_prob': 0.108424,
   'sentiment': 0}],
 'log_id': 3091101141499855881,
 'text': '何弘恺的脚太臭了'}

In [7]:
finished = open('./final_result.txt',encoding='utf-8').readlines()
finished = list(set([i.strip().split('+')[0] for i in finished]))

In [8]:
weibo_list = [i for i in glob.glob('./weibo/data/*.txt') if 'finished' in i]

In [9]:
weibo_list = [i for i in weibo_list if i.split('\\')[-1].split('_')[0] not in finished]

In [10]:
news_list = open('../weibo/stk_list.txt',encoding='utf-8').readlines()
news_list = [i.strip().replace('*','') for i in news_list]

In [11]:
weibo_list = [i for i in weibo_list if i.split('\\')[-1].split('_')[0] in news_list]

In [12]:
weibos = pd.read_csv('../data/final_result.txt',sep='+')

In [13]:
stk_list = list(set(list(weibos['stk'])))

In [14]:
already_baidu = list(set([i.split('+')[0] for i in open('final_result.txt',encoding='utf-8').readlines()]))

In [23]:
baidu_list = open('../baidu/bd_news.txt',encoding='utf-8').readlines()
baidu_list = [i.strip() for i in baidu_list]
#baidu_list = [i for i in baidu_list if i.split(' ')[1] in stk_list]
#baidu_list = [i for i in baidu_list if i.split(' ')[1] not in already_baidu]

In [ ]:
jj = 0
gg = 0
#result = {'stk':[],'text':[],'date':[],'sentiment':[],'confidence':[],'negative_prob':[],'positive_prob':[]}
with io.open('final_result.txt',mode='a',encoding='utf-8') as f:
    for stk in weibo_list:
        stk_name = stk.split('\\')[1].split('_finished')[0]
        data = open(stk,encoding='utf-8').readlines()
        data = [(i.strip()[2:-2][:-23],i.strip()[2:-2][-19:]) for i in data]
        for i in data:
            text = is_ustr(remove_emoji(i[0]))
            if any([(i in text) for i in list(stk_name)]):
                date = i[1][:-9]
                sentiment_result = client.sentimentClassify(text)
                gg = 0
                while ('items' not in sentiment_result) & (gg <= 20):
                    sentiment_result = client.sentimentClassify(text)
                    gg+=1
                sentiment = sentiment_result['items'][0]['sentiment']
                confidence = sentiment_result['items'][0]['confidence']
                negative_prob = sentiment_result['items'][0]['negative_prob']
                positive_prob = sentiment_result['items'][0]['positive_prob']
                f.write('{}+{}+{}+{}+{}+{}+{}'.format(stk_name,text.replace('+',''),date,sentiment,confidence,negative_prob,positive_prob))
                f.write('\n')
                #result['stk'].append(stk_name)
                #result['text'].append(text)
                #result['date'].append(date)
                #result['sentiment'].append(sentiment)
                #result['confidence'].append(confidence)
                #result['negative_prob'].append(negative_prob)
                #result['positive_prob'].append(positive_prob)
                gg += 1
                if gg % 50 == 0:
                    print('{} texts finished'.format(gg))
        jj += 1
        if jj % 10 == 0:
            print('stocks finished {}/{}'.format(jj,len(weibo_list)))

In [ ]:
i = baidu_list[0]

In [ ]:
' '.join(i.split(' ')[2:-2])

In [ ]:
i.split(' ')[-2][1:]

In [ ]:
gg

In [25]:
len(baidu_list)

38224

In [26]:
baidu_list[:50]

['2770 科迪乳业 "科迪乳业盘中异动 股价大涨5.03%" "2018-07-17 14:01:41.274596"',
 '2770 科迪乳业 "科迪乳业宣布推行预付款策略 加快经销商资金回笼速度" "2018-07-17 13:01:41.275753"',
 '2770 科迪乳业 "老李看盘:奶业全面振兴暗地发酵 科迪乳业逆势二连板" "2018-07-16 21:50:00"',
 '2770 科迪乳业 "“奶业振兴”持续发酵 科迪乳业涨停实现“二连板”" "2018-07-16 21:26:00"',
 '2770 科迪乳业 "昨日涨停板块大涨 科迪乳业涨停" "2018-07-16 14:05:00"',
 '2770 科迪乳业 "科迪乳业:严重超跌快速封板 即将复制连板走势" "2018-07-15 19:40:00"',
 '2770 科迪乳业 "科迪乳业涨停 领涨乳业板块" "2018-07-13 13:38:00"',
 '2770 科迪乳业 话题:食品饮料板块表现强势科迪乳业涨停 "2018-07-13 13:35:00"',
 '2770 科迪乳业 "国产乳业板块强势 科迪乳业涨停" "2018-07-13 13:35:00"',
 '2770 科迪乳业 "“奶业振兴”势在必行 科迪乳业领衔多只乳业个股大涨" "2018-07-13 12:01:00"',
 '2770 科迪乳业 "乳业板块异动拉升 科迪乳业涨停皇氏集团上涨" "2018-07-13 11:18:00"',
 '2770 科迪乳业 "快讯:乳业板块异动拉升 科迪乳业涨停" "2018-07-13 11:13:00"',
 '2770 科迪乳业 "科迪乳业盘中异动 股价拉升5.42%" "2018-07-13 11:05:00"',
 '2770 科迪乳业 "科迪乳业:辽宁推出二胎鼓励政策 科迪乳业涨停" "2018-07-12 23:28:00"',
 '2770 科迪乳业 "科迪乳业盘中异动 临近午盘大幅拉升5.24%" "2018-07-12 11:29:00"',
 '2770 科迪乳业 "科迪乳业现1笔大宗交易 共成交8,157.73万元" "2018-07-11 17:00:00"',
 '2770 科迪乳业 "大股东质押触及平仓线 科迪乳业“自救路”难走" 

In [37]:
ggg = 0
#result = {'stk':[],'text':[],'date':[],'sentiment':[],'confidence':[],'negative_prob':[],'positive_prob':[]}
with io.open('final_result.txt',mode='a',encoding='utf-8') as f:
    for i in baidu_list:
        stk_name = i.split(' ')[1].replace('*','')
        text = ' '.join(i.split(' ')[2:-2])
        text = is_ustr(remove_emoji(text))
        if any([(i in text) for i in list(stk_name)]):
            date = i.split(' ')[-2][1:]
            sentiment_result = client.sentimentClassify(text)
            gg = 0
            while ('items' not in sentiment_result) & (gg <= 20):
                sentiment_result = client.sentimentClassify(text)
                gg+=1
            sentiment = sentiment_result['items'][0]['sentiment']
            confidence = sentiment_result['items'][0]['confidence']
            negative_prob = sentiment_result['items'][0]['negative_prob']
            positive_prob = sentiment_result['items'][0]['positive_prob']
            f.write('{}+{}+{}+{}+{}+{}+{}'.format(stk_name,text.replace('+',''),date,sentiment,confidence,negative_prob,positive_prob))
            f.write('\n')
            #result['stk'].append(stk_name)
            #result['text'].append(text)
            #result['date'].append(date)
            #result['sentiment'].append(sentiment)
            #result['confidence'].append(confidence)
            #result['negative_prob'].append(negative_prob)
            #result['positive_prob'].append(positive_prob)
            ggg += 1
            #if ggg % 50 == 0:
            print('{} texts finished'.format(ggg))

1 texts finished
2 texts finished
3 texts finished
4 texts finished
5 texts finished
6 texts finished
7 texts finished
8 texts finished
9 texts finished
10 texts finished
11 texts finished
12 texts finished
13 texts finished
14 texts finished
15 texts finished
16 texts finished
17 texts finished
18 texts finished
19 texts finished
20 texts finished
21 texts finished
22 texts finished
23 texts finished
24 texts finished
25 texts finished
26 texts finished
27 texts finished
28 texts finished
29 texts finished
30 texts finished
31 texts finished
32 texts finished
33 texts finished
34 texts finished
35 texts finished
36 texts finished
37 texts finished
38 texts finished
39 texts finished
40 texts finished
41 texts finished
42 texts finished
43 texts finished
44 texts finished
45 texts finished
46 texts finished
47 texts finished
48 texts finished
49 texts finished
50 texts finished
51 texts finished
52 texts finished
53 texts finished
54 texts finished
55 texts finished
56 texts finished
5